Cargamos el csv que contiene cada imagen con sus yemas ground truth separadas.  
Si la imagen tiene dos o mas filas, entonces se trata de una multiyema.  
Creamos un nuevo csv llamado multi_instance_dataset.csv

In [1]:
import pandas as pd
import numpy as np
import os, cv2
import PIL
import matplotlib.pyplot as plt

In [11]:
single_instance_csv_path = 'D:\\temp\\dataset_resize\\detection_dataset.csv'

In [12]:
single_instance_csv = pd.read_csv(single_instance_csv_path)

In [13]:
single_instance_csv.head(10)

,Unnamed: 0,imageOrigin,mask
0,0,0001.jpg,0001_0001_mask.png
1,1,0002.jpg,0002_0032_mask.png
2,2,0003.jpg,0003_0056_mask.png
3,3,0004.jpg,0004_0121_mask.png
4,4,0005.jpg,0005_0163_mask.png
5,5,0006.jpg,0006_0189_mask.png
6,6,0007.jpg,0007_0208_mask.png
7,7,0008.jpg,0008_0227_mask.png
8,8,0008.jpg,0008_0228_mask.png
9,9,0009.jpg,0009_0244_mask.png


In [14]:
multi_instance_subset = single_instance_csv.loc[single_instance_csv.duplicated(['imageOrigin'], keep=False), :]

In [15]:
multi_instance_image_array = pd.unique(multi_instance_subset.drop(columns='Unnamed: 0').imageOrigin)

In [16]:
multi_instance_mask_array = ['mask_' + i[:-3] + 'png' for i in  multi_instance_image_array] 

In [18]:
final_multinstance_csv = pd.DataFrame({
    'imageOrigin' : multi_instance_image_array,
    'mask' : multi_instance_mask_array
})
final_multinstance_csv.to_csv('multi_instance_dataset.csv')

Etapa de inferencia sobre imagenes multiyema

In [2]:
from utils import DataGeneratorMobileNetKeras
from keras.models import load_model

Using TensorFlow backend.


In [3]:
MODEL_NAME = "11FCMN8rmsprop_lr0.0001_prep_keras_dp0.001_ep200.h5"
MODEL_PATH = os.path.join('..' , 'output', 'models', MODEL_NAME)
IMAGES_PATH = 'D:\\temp\\dataset_resize\\images_resize'
MASK_PATH = "D:\\temp\\dataset_resize\\masks_resize"

In [4]:
csv = pd.read_csv("multi_instance_dataset.csv")

In [5]:
labels = dict(zip(csv['imageOrigin'].tolist(), csv['mask'].tolist()))

In [6]:
model = load_model(MODEL_PATH)














Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




In [7]:
valid_generator = DataGeneratorMobileNetKeras(batch_size=1,img_path=IMAGES_PATH, labels=labels,
            list_IDs=csv['imageOrigin'].tolist(),n_channels=3, n_channels_label=1,shuffle=False,mask_path=MASK_PATH)

In [9]:
prediction = model.predict_generator(generator=valid_generator, use_multiprocessing=False, verbose=True)

43/43 [==============================] - 137s 3s/step


In [12]:
np.save('multi_instance_test_masks', prediction)

In [14]:
def save_image(image, out_path, image_name):
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    cv2.imwrite(os.path.join(out_path, image_name),image)

In [18]:
img_name_list = csv['imageOrigin'].tolist()
for i in range(prediction.shape[0]):
    save_image(prediction[i,:,:,0] * 255, os.path.join('.', 'multi_predictions'), 'norm_pred_' + img_name_list[i])